<a href="https://colab.research.google.com/github/KornelWitkowski/Deep-Q-Learning-with-Tensorflow/blob/main/Cart_Pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cart Pole

In [1]:
%%bash

apt-get install swig

git clone https://github.com/pybox2d/pybox2d
cd pybox2d
python setup.py build
python setup.py install

apt-get install -y xvfb

pip install \
    gym==0.21 \
    gym[box2d]==0.21 \
    pyglet==1.5.27 \
    pyvirtualdisplay

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,447 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 

In [2]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start();

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam


class DeepQLearningModel:
  def __init__(self, observation_size, hidden_size, action_size, learning_rate=0.001):

    self.model = Sequential([Dense(observation_size, activation="relu"),
                             Dense(hidden_size, activation="relu"),
                             Dense(hidden_size, activation="relu"),
                             Dense(action_size)])
    self.compile()
    
  def compile(self, learning_rate=0.001):  
    self.model.compile(loss="mse",
                       optimizer=Adam(learning_rate=learning_rate))
    
  def fit(self, x, y):
    history = self.model.fit(x, y, epochs=1, verbose=0)
    loss =  history.history["loss"][0]
    return loss

In [4]:
import numpy as np

def epsilon_greedy_policy(state, environment, model, epsilon=0.0):

  if np.random.random() < epsilon:
    action = environment.action_space.sample()
  else:
    q_values = model(tf.expand_dims(state, axis=0))
    action = tf.math.argmax(q_values, axis=1)
    action = int(action)
  return action

In [5]:
from collections import deque, namedtuple
import random

class ReplayBuffer:

  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)

  def __len__(self):
    return len(self.buffer)

  def append(self, experience):
    self.buffer.append(experience)

  def sample(self, batch_size):
    return random.sample(self.buffer, batch_size)

In [6]:
import random
from collections import deque

class ReplayBuffer:

  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)

  def __len__(self):
    return len(self.buffer)

  def append(self, experience):
    self.buffer.append(experience)

  def sample(self, batch_size):
    return random.sample(self.buffer, batch_size)

In [7]:
import gym
from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit

def create_gym_environment(name):
  environment = gym.make(name)
  environment = TimeLimit(environment, max_episode_steps=400)
  environment = RecordVideo(environment, video_folder='./recored_episodes', episode_trigger=lambda x: x % 50 == 0)
  environment = RecordEpisodeStatistics(environment)

  return environment

In [8]:
import copy
import pandas as pd

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit

import gym

class DeepQLearningAlgorithm:

    def __init__(self, environment_name="CartPole-v1", policy=epsilon_greedy_policy, capacity=100_000,
                batch_size=256, learning_rate=1e-3, hidden_size=128, gamma=0.99, epsilon_start=1.0, epsilon_end=0.15,
                epsilon_last_episode=600, samples_per_epoch=2_048, q_net_update_rate=10):
    
      self.environment = create_gym_environment(environment_name)
      observation_size = self.environment.observation_space.shape[0]
      actions_size = self.environment.action_space.n

      self.q_net = DeepQLearningModel(observation_size, hidden_size, actions_size, learning_rate)
      self.target_q_net = copy.deepcopy(self.q_net.model)

      self.policy = policy
      self.buffer = ReplayBuffer(capacity=capacity)

       
      self.current_epoch = 0


      # hyperparameters
      self.batch_size = batch_size
      self.learning_rate = learning_rate
      self.gamma = 0.99
      self.epsilon_start = epsilon_start
      self.epsilon_end = epsilon_end
      self.epsilon_last_episode = epsilon_last_episode
      self.samples_per_epoch = samples_per_epoch
      self.q_net_update_rate = 10


      while len(self.buffer) < self.samples_per_epoch:
        self.play_episode(epsilon=0)
        
    def play_episode(self, policy=None, epsilon=0.):
      state = self.environment.reset()
      done = False

      while not done:
        if policy:
          action = policy(state, self.environment, self.q_net.model, epsilon=epsilon)
        else:
          action = self.environment.action_space.sample()

        next_state, reward, done, info = self.environment.step(action)
        exp = (state, action, reward, done, next_state)
        self.buffer.append(exp)
        state = next_state

    def get_batch(self):
      sample = pd.DataFrame(self.buffer.sample(self.batch_size))

      state = np.stack(sample[0].values)
      action = np.stack(sample[1].values)
      reward = np.stack(sample[2].values)
      done = np.stack(sample[3].values)
      next_state = np.stack(sample[4].values)

      return state, action, reward, done, next_state


    def train_step(self):
      states, actions, rewards, dones, next_states = self.get_batch()

      state_action_values = tf.gather(self.q_net.model(states), actions, axis=1, batch_dims=1)

      next_action_values = tf.math.reduce_max(self.target_q_net(next_states), axis=1)
      next_action_values = next_action_values.numpy()
      next_action_values[dones] = 0.0

      expected_state_action_values = rewards + self.gamma * next_action_values

      q_net_predictions = self.q_net.model(states).numpy()
      q_net_predictions[range(self.batch_size), actions] = expected_state_action_values

      loss = self.q_net.fit(states, q_net_predictions)

      return loss

      
    def training_epoch_end(self):

        epsilon = max(self.epsilon_end,
                      self.epsilon_start - self.current_epoch / self.epsilon_last_episode)

        self.play_episode(policy=self.policy, epsilon=epsilon)

        if self.current_epoch % self.q_net_update_rate == 0:
          self.target_q_net = copy.deepcopy(self.q_net.model)

        return
        
    def train(self, epochs):

      for i in range(epochs):

        loss = 0

        for _ in range(self.samples_per_epoch//self.batch_size):  
          loss += self.train_step()

        self.training_epoch_end()

        if self.current_epoch % 50 == 0:
          returns = list(self.environment.return_queue)[-1]
          print(f"Epoch: {self.current_epoch}, loss: {loss}, hp_metric: {tf.math.reduce_mean(returns)}")

        self.current_epoch += 1


In [9]:
!rm -r /content/recored_episodes

algorithm =  DeepQLearningAlgorithm()

In [10]:
algorithm.train(2000)

Epoch: 0, loss: 1.1428114539012313, hp_metric: 18.0
Epoch: 50, loss: 0.8514621183276176, hp_metric: 46.0
Epoch: 100, loss: 1.6077680066227913, hp_metric: 13.0
Epoch: 150, loss: 2.711870774626732, hp_metric: 13.0
Epoch: 200, loss: 3.040138378739357, hp_metric: 85.0
Epoch: 250, loss: 5.279157966375351, hp_metric: 20.0
Epoch: 300, loss: 7.93424379825592, hp_metric: 25.0
Epoch: 350, loss: 13.34866014122963, hp_metric: 286.0
Epoch: 400, loss: 21.881786227226257, hp_metric: 103.0
Epoch: 450, loss: 21.731561541557312, hp_metric: 200.0
Epoch: 500, loss: 20.346328556537628, hp_metric: 24.0
Epoch: 550, loss: 23.479199945926666, hp_metric: 120.0
Epoch: 600, loss: 38.78479528427124, hp_metric: 15.0
Epoch: 650, loss: 30.813915491104126, hp_metric: 177.0
Epoch: 700, loss: 42.59800082445145, hp_metric: 165.0
Epoch: 750, loss: 38.58635640144348, hp_metric: 332.0
Epoch: 800, loss: 55.14307129383087, hp_metric: 56.0
Epoch: 850, loss: 27.538523375988007, hp_metric: 254.0
Epoch: 900, loss: 25.869384229183